[![Dataflowr](https://raw.githubusercontent.com/dataflowr/website/master/_assets/dataflowr_logo.png)](https://dataflowr.github.io/website/)

# Module 2: PyTorch tensors and automatic differentiation

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=103)

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np

In [ ]:
torch.__version__

Tensors are used to encode the signal to process, but also the internal states and parameters of models.

**Manipulating data through this constrained structure allows to use CPUs and GPUs at peak performance.**

Construct a 3x5 matrix, uninitialized:

In [3]:
x = torch.empty(3,5)
print(x.dtype)
print(x)

torch.float32
tensor([[-8.8648e-19,  4.5556e-41, -1.3909e-37,  4.5556e-41, -3.1079e-19],
        [ 4.5556e-41, -1.3910e-37,  4.5556e-41, -2.4235e-06,  3.0754e-41],
        [-1.3910e-37,  4.5556e-41, -2.4234e-06,  3.0754e-41, -2.7653e-19]])


If you got an error this [stackoverflow link](https://stackoverflow.com/questions/50617917/overflow-when-unpacking-long-pytorch) might be useful...

In [4]:
x = torch.randn(3,5)
print(x)

tensor([[ 0.8918, -0.3444,  0.3130, -0.6937,  1.4565],
        [-0.5341,  1.1907,  0.0455, -2.1627, -1.1446],
        [ 0.1281,  0.3317,  0.2186, -0.2452, -1.0070]])


In [5]:
print(x.size())

torch.Size([3, 5])


torch.Size is in fact a [tuple](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences), so it supports the same operations.

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=272)

In [6]:
x.size()[1]

5

In [7]:
x.size() == (3,5)

True

### Bridge to numpy

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=325)

In [8]:
y = x.numpy()
print(y)

[[ 0.8917917  -0.34437996  0.31297627 -0.6937169   1.4564596 ]
 [-0.5340678   1.1907384   0.0454716  -2.1627245  -1.1446208 ]
 [ 0.12805997  0.33172658  0.21858421 -0.245166   -1.0070175 ]]


In [9]:
a = np.ones(5)
b = torch.from_numpy(a)
print(a.dtype)
print(b)

float64
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [10]:
c = b.long()
print(c.dtype, c)
print(b.dtype, b)

torch.int64 tensor([1, 1, 1, 1, 1])
torch.float64 tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [11]:
xr = torch.randn(3, 5)
print(xr.dtype, xr)

torch.float32 tensor([[-0.0863,  0.5858,  1.3539,  0.9865,  1.3506],
        [ 1.3485, -2.2523,  0.4640,  1.8543,  0.5660],
        [-0.5236, -0.4916, -0.7430, -2.0052,  0.2551]])


In [12]:
resb = xr + b
resb

tensor([[ 0.9137,  1.5858,  2.3539,  1.9865,  2.3506],
        [ 2.3485, -1.2523,  1.4640,  2.8543,  1.5660],
        [ 0.4764,  0.5084,  0.2570, -1.0052,  1.2551]], dtype=torch.float64)

In [13]:
resc = xr + c
resc

tensor([[ 0.9137,  1.5858,  2.3539,  1.9865,  2.3506],
        [ 2.3485, -1.2523,  1.4640,  2.8543,  1.5660],
        [ 0.4764,  0.5084,  0.2570, -1.0052,  1.2551]])

Be careful with types!

In [14]:
resb == resc

tensor([[False,  True, False,  True,  True],
        [ True,  True, False,  True, False],
        [ True,  True,  True,  True, False]])

In [15]:
torch.set_printoptions(precision=10)

In [16]:
resb[0,1]

tensor(1.5858143568, dtype=torch.float64)

In [17]:
resc[0,1]

tensor(1.5858143568)

In [18]:
resc[0,1].dtype

torch.float32

In [19]:
xr[0,1]

tensor(0.5858143568)

In [20]:
torch.set_printoptions(precision=4)

### [Broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=670)

Broadcasting automagically expands dimensions by replicating coefficients, when it is necessary to perform operations.

1. If one of the tensors has fewer dimensions than the other, it is reshaped by adding as many dimensions of size 1 as necessary in the front; then
2. for every mismatch, if one of the two tensor is of size one, it is expanded along this axis by replicating  coefficients.

If there is a tensor size mismatch for one of the dimension and neither of them is one, the operation fails.

In [21]:
A = torch.tensor([[1.], [2.], [3.], [4.]])
print(A.size())
B = torch.tensor([[5., -5., 5., -5., 5.]])
print(B.size())
C = A + B

torch.Size([4, 1])
torch.Size([1, 5])


In [22]:
C

tensor([[ 6., -4.,  6., -4.,  6.],
        [ 7., -3.,  7., -3.,  7.],
        [ 8., -2.,  8., -2.,  8.],
        [ 9., -1.,  9., -1.,  9.]])

The original (column-)vector
\begin{eqnarray*}
A = \left( \begin{array}{c}
1\\
2\\
3\\
4\\
\end{array}\right)
\end{eqnarray*}
is transformed into the matrix 
\begin{eqnarray*}
A = \left( \begin{array}{ccccc}
1&1&1&1&1\\
2&2&2&2&2\\
3&3&3&3&3\\
4&4&4&4&4
\end{array}\right)
\end{eqnarray*}
and the original (row-)vector
\begin{eqnarray*}
B = (5,-5,5,-5,5)
\end{eqnarray*}
is transformed into the matrix
\begin{eqnarray*}
B = \left( \begin{array}{ccccc}
5&-5&5&-5&5\\
5&-5&5&-5&5\\
5&-5&5&-5&5\\
5&-5&5&-5&5
\end{array}\right)
\end{eqnarray*}
so that summing these matrices gives:
\begin{eqnarray*}
A+B = \left( \begin{array}{ccccc}
6&-4&6&-4&6\\
7&-3&7&-3&7\\
8&-2&8&-2&8\\
9&-1&9&-1&9
\end{array}\right)
\end{eqnarray*}

The original (column-)vector
$$\begin{array}{ccc}A = \left( \begin{array}{c}1\\2\\3\\4\\\end{array}\right)\end{array}
$$
is transformed into the matrix 
$$\begin{array}{}
A = \left( \begin{array}{ccccc}
1&1&1&1&1\\
2&2&2&2&2\\
3&3&3&3&3\\
4&4&4&4&4
\end{array}\right)
\end{array}
$$
and the original (row-)vector
\begin{array}{}
B = (5,-5,5,-5,5)
\end{array}
is transformed into the matrix
$$\begin{array}{}
B = \left( \begin{array}{ccccc}
5&-5&5&-5&5\\
5&-5&5&-5&5\\
5&-5&5&-5&5\\
5&-5&5&-5&5
\end{array}\right)
\end{array}
$$
so that summing these matrices gives:
$$\begin{array}{}
A+B = \left( \begin{array}{ccccc}
6&-4&6&-4&6\\
7&-3&7&-3&7\\
8&-2&8&-2&8\\
9&-1&9&-1&9
\end{array}\right)
\end{array}
$$

### In-place modification

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=875)

In [ ]:
x

In [ ]:
xr

In [ ]:
print(x+xr)

In [ ]:
x.add_(xr)
print(x)

Any operation that mutates a tensor in-place is post-fixed with an ```_```

For example: ```x.fill_(y)```, ```x.t_()```, will change ```x```.

In [23]:
print(x.t())

tensor([[ 0.8918, -0.5341,  0.1281],
        [-0.3444,  1.1907,  0.3317],
        [ 0.3130,  0.0455,  0.2186],
        [-0.6937, -2.1627, -0.2452],
        [ 1.4565, -1.1446, -1.0070]])


In [24]:
x.t_()
print(x)

tensor([[ 0.8918, -0.5341,  0.1281],
        [-0.3444,  1.1907,  0.3317],
        [ 0.3130,  0.0455,  0.2186],
        [-0.6937, -2.1627, -0.2452],
        [ 1.4565, -1.1446, -1.0070]])


### Shared memory

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=990)

Also be careful, changing the torch tensor modify the numpy array and vice-versa...

This is explained in the PyTorch documentation [here](https://pytorch.org/docs/stable/torch.html#torch.from_numpy):
The returned tensor by `torch.from_numpy` and ndarray share the same memory. Modifications to the tensor will be reflected in the ndarray and vice versa. 

In [25]:
a = np.ones(5)
b = torch.from_numpy(a)
print(b)

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [26]:
a[2] = 0
print(b)

tensor([1., 1., 0., 1., 1.], dtype=torch.float64)


In [27]:
b[3] = 5
print(a)

[1. 1. 0. 5. 1.]


### Cuda

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=1120)

In [28]:
torch.cuda.is_available()

True

In [29]:
#device = torch.device('cpu')
device = torch.device('cuda') # Uncomment this to run on GPU

In [30]:
x.device

device(type='cpu')

In [31]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z,z.type())
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([[ 1.8918,  0.4659,  1.1281],
        [ 0.6556,  2.1907,  1.3317],
        [ 1.3130,  1.0455,  1.2186],
        [ 0.3063, -1.1627,  0.7548],
        [ 2.4565, -0.1446, -0.0070]], device='cuda:0') torch.cuda.FloatTensor
tensor([[ 1.8918,  0.4659,  1.1281],
        [ 0.6556,  2.1907,  1.3317],
        [ 1.3130,  1.0455,  1.2186],
        [ 0.3063, -1.1627,  0.7548],
        [ 2.4565, -0.1446, -0.0070]], dtype=torch.float64)


In [32]:
x = torch.randn(1)
x = x.to(device)

In [33]:
x.device

device(type='cuda', index=0)

In [34]:
# the following line is only useful if CUDA is available
x = x.data
print(x)
print(x.item())
print(x.cpu().numpy())

tensor([1.9704], device='cuda:0')
1.97038996219635
[1.97039]


# Simple interfaces to standard image data-bases

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=1354)

An example, the [CIFAR10](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.CIFAR10) dataset.

In [35]:
import torchvision

data_dir = '/home/andy/00_workspace/dl/notebooks/Module2/data'

cifar = torchvision.datasets.CIFAR10(data_dir, train = True, download = True)
cifar.data.shape

/home/andy/anaconda3/envs/py37/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/andy/anaconda3/envs/py37/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /home/andy/00_workspace/dl/notebooks/Module2/data/cifar-10-python.tar.gz to /home/andy/00_workspace/dl/notebooks/Module2/data


(50000, 32, 32, 3)

Documentation about the [`permute`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.permute) operation.

In [36]:
x = torch.from_numpy(cifar.data).permute(0,3,1,2).float()
x = x / 255
print(x.type(), x.size(), x.min().item(), x.max().item())

torch.FloatTensor torch.Size([50000, 3, 32, 32]) 0.0 1.0


Documentation about the [`narrow(input, dim, start, length)`](https://pytorch.org/docs/stable/torch.html#torch.narrow) operation.

In [37]:
# Narrows to the first images, converts to float
x = torch.narrow(x, 0, 0, 48)

In [38]:
x.shape

torch.Size([48, 3, 32, 32])

In [ ]:
# Showing images
def show(img):
    npimg = img.numpy()
    plt.figure(figsize=(20,10))
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    
show(torchvision.utils.make_grid(x, nrow = 12))

In [ ]:
# Kills the green and blue channels
x.narrow(1, 1, 2).fill_(0)
show(torchvision.utils.make_grid(x, nrow = 12))

# Autograd: automatic differentiation

[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=40)

When executing tensor operations, PyTorch can automatically construct on-the-fly the graph of operations to compute the gradient of any quantity with respect to any tensor involved.

To be more concrete, we introduce the following example: we consider parameters $w\in \mathbb{R}$ and $b\in \mathbb{R}$ with the corresponding function:
\begin{eqnarray*}
\ell = \left(\exp(wx+b) - y^* \right)^2
\end{eqnarray*}

Our goal here, will be to compute the following partial derivatives:
\begin{eqnarray*}
\frac{\partial \ell}{\partial w}\mbox{ and, }\frac{\partial \ell}{\partial b}.
\end{eqnarray*}

The reason for doing this will be clear when you will solve the practicals for this lesson!

You can decompose this function as a composition of basic operations. This is call the forward pass on the graph of operations.
![backprop1](https://dataflowr.github.io/notebooks/Module2/img/backprop1.png)

Let say we start with our model in `numpy`:

In [39]:
w = np.array([0.5])
b = np.array([2])
xx = np.array([0.5])#np.arange(0,1.5,.5)

transform these into `tensor`:

In [40]:
xx_t = torch.from_numpy(xx)
w_t = torch.from_numpy(w)
b_t = torch.from_numpy(b)

[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=224)

A `tensor` has a Boolean field `requires_grad`, set to `False` by default, which states if PyTorch should build the graph of operations so that gradients with respect to it can be computed.

In [41]:
w_t.requires_grad

False

We want to take derivative with respect to $w$ so we change this value:

In [42]:
w_t.requires_grad_(True)

tensor([0.5000], dtype=torch.float64, requires_grad=True)

We want to do the same thing for $b$ but the following line will produce an error!

In [43]:
b_t.requires_grad_(True)

RuntimeError: only Tensors of floating point dtype can require gradients

Reading the error message should allow you to correct the mistake!

In [44]:
dtype = torch.float64

In [45]:
b_t = b_t.type(dtype)

In [46]:
b_t.requires_grad_(True)

tensor([2.], dtype=torch.float64, requires_grad=True)

[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=404)

We now compute the function:

In [47]:
def fun(x,ystar):
    y = torch.exp(w_t*x+b_t)
    print(y)
    return torch.sum((y-ystar)**2)

ystar_t = torch.randn_like(xx_t)
l_t = fun(xx_t,ystar_t)

tensor([9.4877], dtype=torch.float64, grad_fn=<ExpBackward0>)


In [48]:
l_t

tensor(69.1070, dtype=torch.float64, grad_fn=<SumBackward0>)

In [49]:
l_t.requires_grad

True

After the computation is finished, i.e. *forward pass*, you can call ```.backward()``` and have all the gradients computed automatically.

In [50]:
print(w_t.grad)

None


In [51]:
l_t.backward()

In [52]:
print(w_t.grad)
print(b_t.grad)

tensor([78.8721], dtype=torch.float64)
tensor([157.7442], dtype=torch.float64)


[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=545)

Let's try to understand these numbers...

![backprop2](https://dataflowr.github.io/notebooks/Module2/img/backprop2.png)

In [53]:
yy_t = torch.exp(w_t*xx_t+b_t)
print(torch.sum(2*(yy_t-ystar_t)*yy_t*xx_t))
print(torch.sum(2*(yy_t-ystar_t)*yy_t))

tensor(78.8721, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(157.7442, dtype=torch.float64, grad_fn=<SumBackward0>)


`tensor.backward()` accumulates the gradients in  the `grad` fields  of tensors.

In [54]:
l_t = fun(xx_t,ystar_t)
l_t.backward()

tensor([9.4877], dtype=torch.float64, grad_fn=<ExpBackward0>)


In [55]:
print(w_t.grad)
print(b_t.grad)

tensor([157.7442], dtype=torch.float64)
tensor([315.4885], dtype=torch.float64)


By default, `backward` deletes the computational graph when it is used so that you will get an error below:

In [56]:
l_t.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [57]:
# Manually zero the gradients
w_t.grad.data.zero_()
b_t.grad.data.zero_()
l_t = fun(xx_t,ystar_t)
l_t.backward(retain_graph=True)
l_t.backward()
print(w_t.grad)
print(b_t.grad)

tensor([9.4877], dtype=torch.float64, grad_fn=<ExpBackward0>)
tensor([157.7442], dtype=torch.float64)
tensor([315.4885], dtype=torch.float64)


The gradients must be set to zero manually. Otherwise they will cumulate across several _.backward()_ calls. 
This accumulating behavior is desirable in particular to compute the gradient of a loss summed over several “mini-batches,” or the gradient of a sum of losses.

[![Dataflowr](https://raw.githubusercontent.com/dataflowr/website/master/_assets/dataflowr_logo.png)](https://dataflowr.github.io/website/)